0. Install and Import Dependencies 

In [3]:
!pip install --user mediapipe opencv-python pandas scikit-learn

  Using cached mediapipe-0.8.9.1-cp310-cp310-win_amd64.whl (48.5 MB)
  Using cached scikit_learn-1.0.1-cp310-cp310-win_amd64.whl
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached matplotlib-3.5.1-cp310-cp310-win_amd64.whl (7.2 MB)
  Using cached opencv_contrib_python-4.5.4.60-cp310-cp310-win_amd64.whl (42.0 MB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached scipy-1.7.3-cp310-cp310-win_amd64.whl (34.3 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


In [5]:
!pip install opencv-python

In [4]:
import mediapipe as mp
import cv2

In [80]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
print(type(mp_drawing))

<class 'module'>


1. Making some detections

In [81]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
#         print(type(ret))
#         print(type(frame))
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        print(type(results.face_landmarks))
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'mediapipe.framework.formats.la

In [82]:
results.pose_landmarks.landmark[0].visibility
# 0 to 1 for pose_landmark 0 for pose_landmark as default

0.9997382760047913

2. Capture Landmarks and export to CSV

In [83]:
import csv
import os
import numpy as np

In [84]:
num_coords = len(results.face_landmarks.landmark)+len(results.pose_landmarks.landmark)
num_coords

501

In [85]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
# landmarks

In [9]:
with open('coords.csv', mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [34]:
class_name = 'nervous'

In [35]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()   
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
          
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            # beacause len(row) should be equal to 1+501*4
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

3. Train Custom Model Using Scikit Learn

3.1 Read in collected Data and Process

In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [98]:
df = pd.read_csv('coords.csv')
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,neutral,0.546452,0.487204,-0.671844,0.999969,0.570894,0.444538,-0.629380,0.999920,0.587234,...,0.002947,0,0.597113,0.456792,0.016318,0,0.601081,0.454127,0.016751,0
1,neutral,0.546831,0.487861,-0.911877,0.999966,0.570895,0.445072,-0.859747,0.999912,0.587232,...,0.001640,0,0.594488,0.456873,0.015285,0,0.598268,0.454815,0.015568,0
2,neutral,0.546961,0.488282,-0.925406,0.999960,0.570888,0.445359,-0.873277,0.999899,0.587201,...,0.002209,0,0.594162,0.457359,0.016216,0,0.597943,0.455162,0.016521,0
3,neutral,0.547003,0.488326,-0.912478,0.999958,0.570868,0.445361,-0.862163,0.999894,0.587158,...,0.002038,0,0.593395,0.457482,0.016057,0,0.597210,0.455246,0.016389,0
4,neutral,0.547067,0.488417,-0.858345,0.999958,0.570831,0.445367,-0.809795,0.999894,0.587057,...,0.002099,0,0.593029,0.457348,0.015909,0,0.596870,0.455114,0.016202,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,nervous,0.537032,0.492420,-0.965761,0.997771,0.573095,0.419747,-0.930252,0.996397,0.591368,...,-0.017497,0,0.613385,0.422952,0.000450,0,0.618671,0.419145,0.000103,0
3400,nervous,0.537349,0.494911,-0.922866,0.997720,0.574228,0.421341,-0.880517,0.996295,0.592929,...,-0.017297,0,0.610985,0.420738,-0.000458,0,0.616154,0.416768,-0.000838,0
3401,nervous,0.538480,0.494792,-0.906077,0.997740,0.576110,0.421236,-0.869272,0.996344,0.594805,...,-0.016817,0,0.609822,0.416777,0.001435,0,0.614888,0.413032,0.001168,0
3402,nervous,0.538846,0.494714,-0.933762,0.997702,0.576592,0.421077,-0.901142,0.996316,0.595389,...,-0.016121,0,0.609697,0.415284,0.002774,0,0.614594,0.411576,0.002663,0


In [99]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,neutral,0.546452,0.487204,-0.671844,0.999969,0.570894,0.444538,-0.629380,0.999920,0.587234,...,0.002947,0,0.597113,0.456792,0.016318,0,0.601081,0.454127,0.016751,0
1,neutral,0.546831,0.487861,-0.911877,0.999966,0.570895,0.445072,-0.859747,0.999912,0.587232,...,0.001640,0,0.594488,0.456873,0.015285,0,0.598268,0.454815,0.015568,0
2,neutral,0.546961,0.488282,-0.925406,0.999960,0.570888,0.445359,-0.873277,0.999899,0.587201,...,0.002209,0,0.594162,0.457359,0.016216,0,0.597943,0.455162,0.016521,0
3,neutral,0.547003,0.488326,-0.912478,0.999958,0.570868,0.445361,-0.862163,0.999894,0.587158,...,0.002038,0,0.593395,0.457482,0.016057,0,0.597210,0.455246,0.016389,0
4,neutral,0.547067,0.488417,-0.858345,0.999958,0.570831,0.445367,-0.809795,0.999894,0.587057,...,0.002099,0,0.593029,0.457348,0.015909,0,0.596870,0.455114,0.016202,0


In [100]:
df[df['class']=='sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1854,sad,0.549684,0.619154,-0.747584,0.999785,0.577158,0.566224,-0.731058,0.999330,0.593059,...,-0.024163,0,0.608750,0.559344,-0.019071,0,0.612568,0.555169,-0.020094,0
1855,sad,0.552640,0.617852,-0.980357,0.999796,0.579197,0.561347,-0.959209,0.999371,0.595457,...,-0.021147,0,0.605435,0.560587,-0.016601,0,0.608869,0.557262,-0.017613,0
1856,sad,0.553804,0.617797,-0.978435,0.999803,0.579826,0.560143,-0.959825,0.999394,0.596205,...,-0.021820,0,0.605601,0.561749,-0.017349,0,0.609003,0.558181,-0.018336,0
1857,sad,0.554398,0.617783,-0.901099,0.999810,0.580121,0.559546,-0.883212,0.999417,0.596644,...,-0.022155,0,0.605647,0.562793,-0.018042,0,0.609054,0.559345,-0.019091,0
1858,sad,0.556676,0.617575,-0.893979,0.999812,0.581749,0.558443,-0.878704,0.999421,0.598321,...,-0.021933,0,0.605073,0.562266,-0.017329,0,0.608417,0.558982,-0.018345,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,sad,0.528994,0.652022,-0.735903,0.999470,0.551718,0.588627,-0.730624,0.998543,0.569683,...,-0.026474,0,0.571786,0.595072,-0.034622,0,0.575817,0.590745,-0.036694,0
2160,sad,0.525237,0.651739,-0.728464,0.999433,0.547606,0.588339,-0.721673,0.998432,0.565417,...,-0.026642,0,0.569420,0.595169,-0.035382,0,0.573370,0.591034,-0.037488,0
2161,sad,0.522258,0.651842,-0.853833,0.999329,0.544046,0.588444,-0.849675,0.998096,0.561831,...,-0.026872,0,0.567394,0.595455,-0.036257,0,0.571485,0.591153,-0.038450,0
2162,sad,0.520243,0.651987,-0.831714,0.999285,0.541441,0.588538,-0.837070,0.997976,0.559254,...,-0.027027,0,0.565054,0.595456,-0.036856,0,0.569076,0.591254,-0.039067,0


In [101]:
X = df.drop ('class',axis=1) #features
y = df['class'] #target value

In [102]:
y

0       neutral
1       neutral
2       neutral
3       neutral
4       neutral
         ...   
3399    nervous
3400    nervous
3401    nervous
3402    nervous
3403    nervous
Name: class, Length: 3404, dtype: object

In [103]:
# creating train and test partition
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1234)
# X_train
# print(type(X_train))
# print(type(X_test))
# print(type(y_train))
# print(type(y_test))

3.2 Train Machine Learning Classification Model

In [104]:
# We will use X_train or y_train values to fit our model i.e train our model
# and we will use X_test and y_test to validate it
from sklearn.pipeline import make_pipeline # for making a machine learning pipeline
from sklearn.preprocessing import StandardScaler # to our pipeline we pass through a preprocessing function called standard scaler
# levraging standart scalar STANDARDIZES the data, subtracts the main and divides it by the standard deviation
# to keep the data on a same plane so that no ideal feature overshadows the other feature
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# Different models to train each and every one of them and get the best model fit for us

In [105]:
# we creating a dictionary
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [106]:
pipelines.values()

dict_values([Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgeclassifier', RidgeClassifier())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])])

In [107]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [108]:
list(pipelines.values())

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])]

In [109]:
# Here we'll train the model
# Firstly we create an empty dictionary
# Then We try and loop through each one of these pipelines and train
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [110]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [111]:
fit_models['rf'].predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype=object)

In [112]:
fit_models['lr'].predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype=object)

In [113]:
fit_models['rc'].predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype='<U9')

In [114]:
fit_models['rc'].predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype='<U9')

3.3 Evaluate and Serialize Model

In [115]:
from sklearn.metrics import accuracy_score # to calculate performace of the models
import pickle # used by data scientist to save their trained model to disk
import cv2

In [116]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) #making a pridiction and storing it
    print(algo, accuracy_score(y_test, yhat)) # to workout how well the yhat model performed against y_test data 

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [117]:
fit_models['rf'].predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype=object)

In [118]:
y_test

2710       angry
268      neutral
842     surprise
114      neutral
1136       shame
          ...   
303      neutral
918     surprise
2104         sad
2782       angry
2443     worried
Name: class, Length: 1022, dtype: object

In [127]:
# Saving our best model random forest classifier
with open('external.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'],f)

4. Make Detections with Model

In [128]:
# add path
with open('external.pkl', 'rb') as f:
    model = pickle.load(f)

In [125]:
 model.predict(X_test)

array(['angry', 'neutral', 'surprise', ..., 'sad', 'angry', 'worried'],
      dtype=object)

In [141]:
# workflow for Detections
# 1. Detect landmarks
# 2. Predict pose(aka class) based on landmark coords
# 3. Render Landmarks and body language pose using OpenCV
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
# When we read our image through open-cv but default the feed we get back is in BGR format
# But in order to work with media pipe we need our frames to be in RGB format
# So we recolor our feed        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # For peformance improvement we set writable flags to false
    # to prevent copying the image data but you're also then able to use the same image for rendering
        image.flags.writeable = False        
    # finally we use holistic model form the with line, using the process method to actually make detections
    # in which we pass our image variable    
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            # flatten() collapses an array into single method
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
#             # beacause len(row) should be equal to 1+501*4
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
        # MAKE DETECTION
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
        # Getting left ear coordinates
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


nervous [4.11179441e-06 5.95418170e-06 6.81245336e-02 5.23998172e-01
 2.39570704e-01 2.41102439e-12 1.72967223e-14 1.54908427e-01
 1.33233324e-02 8.71280215e-13 6.47645235e-05]
nervous [3.84521544e-06 2.41885797e-06 4.85314067e-02 6.39040317e-01
 1.12755932e-01 1.97929937e-12 2.28779611e-14 1.88727836e-01
 1.08688837e-02 4.87389960e-13 6.93608479e-05]


C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


nervous [4.26482514e-06 2.02818670e-06 5.18346709e-02 6.70572079e-01
 1.20050268e-01 2.99588582e-12 2.75613669e-14 1.40745270e-01
 1.67078412e-02 6.32167177e-13 8.35768736e-05]
nervous [3.42817920e-06 1.94569961e-06 3.55225252e-02 5.96542487e-01
 1.30465428e-01 3.30136080e-12 2.39801630e-14 2.24754911e-01
 1.26554920e-02 5.01533975e-13 5.37831701e-05]


C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

nervous [6.55516003e-08 8.81908111e-08 1.02283100e-03 9.91559397e-01
 5.99837712e-05 1.75003104e-17 4.75738560e-19 6.66553123e-03
 6.92008550e-04 8.60791663e-19 9.47391519e-08]
nervous [1.87256252e-07 6.81154291e-08 3.30778793e-04 9.82795149e-01
 2.19824149e-05 7.74137726e-17 7.24698261e-19 1.65407158e-02
 3.11066312e-04 1.30090850e-18 5.20894983e-08]


C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


nervous [3.77901316e-07 7.76698350e-08 3.69171370e-04 9.88960563e-01
 3.68340061e-05 1.56209728e-16 8.55333182e-19 1.02707211e-02
 3.62178293e-04 2.21547566e-18 7.65137406e-08]
nervous [5.48040170e-07 6.92580972e-08 3.36682621e-04 9.75936174e-01
 2.21869503e-05 3.39309971e-16 7.97758114e-19 2.33117087e-02
 3.92438401e-04 3.01194000e-18 1.92311774e-07]


C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\vinay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


nervous [5.59353631e-07 5.46064534e-08 2.11903705e-04 9.75435407e-01
 3.00711399e-05 1.88706434e-16 4.74340427e-19 2.39479536e-02
 3.73936410e-04 1.90446624e-18 1.13695331e-07]
nervous [1.06800782e-06 1.30249136e-07 2.25680371e-04 9.27650143e-01
 1.31297028e-04 4.12311420e-16 1.01974595e-18 7.14306779e-02
 5.60822559e-04 6.66080948e-18 1.80476844e-07]
